In [1]:
import numpy as np
import torch
import torchvision
import torchinfo

# Python basic

In [9]:
#Local variable holder.
class TLocalVar:
  def __init__(self, l):
    self.__dict__= l

In [13]:
def test_f(a):
  b= 10
  print(locals())
  l= TLocalVar(locals())
  print(l)
  # print(l['a'])
  print(l.a,l.b)
  l.a*= 100
  l.b*= 100
  print(l.a,l.b)
  print(a,b)
  locals()['a']*= 100
  print(a,b)
test_f(100)

{'b': 10, 'a': 100}
100 10
10000 1000
100 10
100 10


In [14]:
#Container class to share variables.
class TContainer(object):
  def __init__(self):
    pass
  def __del__(self):
    pass
  def __str__(self):
    return str(self.__dict__)
  def __repr__(self):
    return str(self.__dict__)
  def __iter__(self):
    return self.__dict__.itervalues()
  def items(self):
    return self.__dict__.items()
  def iteritems(self):
    return self.__dict__.iteritems()
  def keys(self):
    return self.__dict__.keys()
  def values(self):
    return self.__dict__.values()
  def __getitem__(self,key):
    return self.__dict__[key]
  def __setitem__(self,key,value):
    self.__dict__[key]= value
  def __delitem__(self,key):
    del self.__dict__[key]
  def __contains__(self,key):
    return key in self.__dict__
  def Cleanup(self):
    keys= self.__dict__.keys()
    for k in keys:
      self.__dict__[k]= None
      del self.__dict__[k]

In [15]:
l= TContainer()

In [16]:
for l.i in range(3):
  print(l.i)

0
1
2


# Torch Tensor

## shape and view

In [2]:
# n_batch= 1
n_batch= 4

In [3]:
x= torch.tensor(np.zeros((n_batch,3,32,32))).float()

In [4]:
x.shape

torch.Size([4, 3, 32, 32])

In [5]:
x[0].numel()

3072

In [6]:
x.view(x.size(0), -1).shape

torch.Size([4, 3072])

In [7]:
l=torch.nn.Linear(x[0].numel(), 10)
l

Linear(in_features=3072, out_features=10, bias=True)

In [8]:
l(x[0].view(-1))

tensor([ 0.0170, -0.0096, -0.0093, -0.0021,  0.0015,  0.0124,  0.0121,  0.0003,
        -0.0003, -0.0002], grad_fn=<AddBackward0>)

In [9]:
l(x.view(x.size(0), -1))

tensor([[ 0.0170, -0.0096, -0.0093, -0.0021,  0.0015,  0.0124,  0.0121,  0.0003,
         -0.0003, -0.0002],
        [ 0.0170, -0.0096, -0.0093, -0.0021,  0.0015,  0.0124,  0.0121,  0.0003,
         -0.0003, -0.0002],
        [ 0.0170, -0.0096, -0.0093, -0.0021,  0.0015,  0.0124,  0.0121,  0.0003,
         -0.0003, -0.0002],
        [ 0.0170, -0.0096, -0.0093, -0.0021,  0.0015,  0.0124,  0.0121,  0.0003,
         -0.0003, -0.0002]], grad_fn=<AddmmBackward>)

## cat (concatenate) and stack

In [10]:
x1= torch.from_numpy(np.array([2,3,4])).float()
x2= torch.from_numpy(np.array([10,22,12])).float()
print(f'x1={x1}')
print(f'x2={x2}')
print(f'torch.cat={torch.cat((x1,x2),axis=0)}')
print(f'torch.stack={torch.stack((x1,x2),axis=0)}')

x1=tensor([2., 3., 4.])
x2=tensor([10., 22., 12.])
torch.cat=tensor([ 2.,  3.,  4., 10., 22., 12.])
torch.stack=tensor([[ 2.,  3.,  4.],
        [10., 22., 12.]])


## view and flatten

In [105]:
n_batch= 64
x= torch.tensor(np.zeros((n_batch,128,6,6))).float()

In [106]:
x.shape

torch.Size([64, 128, 6, 6])

In [107]:
x.view(x.shape[0], -1).shape

torch.Size([64, 4608])

In [108]:
flatten= torch.nn.Flatten()
flatten(x).shape

torch.Size([64, 4608])

In [114]:
torch.flatten(x,1).shape

torch.Size([64, 4608])

## Unsqueeze

In [89]:
x= torch.zeros(3,10,10)
x.shape, x.unsqueeze(0).shape

(torch.Size([3, 10, 10]), torch.Size([1, 3, 10, 10]))

# Modules

## BatchNormal

In [74]:
x= torch.tensor(range(20))/2.0
x= x[torch.randperm(x.shape[0])].reshape(-1,5)
print(f'x= {x} (shape={x.shape})')
bn= torch.nn.BatchNorm1d(5)
# bn.bias.data.fill_(1e-3)
bn.weight.data.fill_(0.)
# bn.bias.data= torch.Tensor([1.,2.,3.,4.,5.])*0.1
# bn.weight.data= torch.Tensor([5.,4.,3.,2.,1.])*0.1
bn.weight.data.fill_(1.)
print(f'bn(x)= {bn(x)}')

x= tensor([[7.5000, 8.0000, 3.5000, 2.0000, 8.5000],
        [9.0000, 5.0000, 4.0000, 6.5000, 1.0000],
        [9.5000, 5.5000, 3.0000, 1.5000, 0.5000],
        [7.0000, 0.0000, 6.0000, 4.5000, 2.5000]]) (shape=torch.Size([4, 5]))
bn(x)= tensor([[-0.7276,  1.1630, -0.5488, -0.8078,  1.6853],
        [ 0.7276,  0.1292, -0.1098,  1.4291, -0.6663],
        [ 1.2127,  0.3015, -0.9879, -1.0563, -0.8231],
        [-1.2127, -1.5937,  1.6465,  0.4350, -0.1960]],
       grad_fn=<NativeBatchNormBackward>)


In [75]:
bn.weight.data, bn.bias.data

(tensor([1., 1., 1., 1., 1.]), tensor([0., 0., 0., 0., 0.]))

In [77]:
torch.mean(x,axis=0), torch.var(x,axis=0)
(x-torch.mean(x,axis=0))/torch.sqrt(torch.var(x,axis=0)) * bn.weight + bn.bias

tensor([[-0.6301,  1.0072, -0.4753, -0.6996,  1.4595],
        [ 0.6301,  0.1119, -0.0951,  1.2377, -0.5770],
        [ 1.0502,  0.2611, -0.8555, -0.9148, -0.7128],
        [-1.0502, -1.3802,  1.4259,  0.3767, -0.1697]], grad_fn=<AddBackward0>)

In [82]:
inn= torch.nn.InstanceNorm1d(5, affine=True)
# inn.bias.data.fill_(1e-3)
# inn.weight.data.fill_(0.)
# inn.bias.data= torch.Tensor([1.,2.,3.,4.,5.])*0.1
# inn.weight.data= torch.Tensor([5.,4.,3.,2.,1.])*0.1
# inn.weight.data.fill_(1.)
print(f'inn(x)= {inn(x)}')

ValueError: InstanceNorm1d returns 0-filled tensor to 2D tensor.This is because InstanceNorm1d reshapes inputs to(1, N * C, ...) from (N, C,...) and this makesvariances 0.

## Max module

In [91]:
x= torch.tensor(range(150))/2.0
x= x[torch.randperm(x.shape[0])].reshape(-1,3,5,5)
x

tensor([[[[59.0000, 17.0000, 55.5000, 56.5000, 58.5000],
          [24.0000, 39.5000, 50.5000, 62.0000, 72.5000],
          [28.0000, 42.5000, 41.0000, 51.0000, 36.5000],
          [54.0000, 70.0000, 35.5000, 20.0000, 28.5000],
          [26.0000, 16.0000,  6.0000, 70.5000, 49.0000]],

         [[19.0000,  0.0000, 23.0000, 48.5000, 14.0000],
          [65.5000, 36.0000,  4.0000, 64.0000, 12.0000],
          [25.0000, 34.0000, 35.0000,  2.5000, 30.5000],
          [61.0000, 54.5000, 25.5000, 60.0000, 69.0000],
          [65.0000,  1.5000, 68.0000, 45.0000, 32.0000]],

         [[19.5000, 31.0000, 57.5000, 13.0000, 46.0000],
          [55.0000, 14.5000, 53.5000,  2.0000, 43.5000],
          [49.5000, 67.5000, 46.5000, 15.5000, 61.5000],
          [34.5000,  3.0000, 10.5000, 44.0000,  1.0000],
          [ 9.0000, 38.5000, 60.5000, 33.5000, 72.0000]]],


        [[[33.0000, 73.5000, 41.5000,  4.5000, 18.5000],
          [31.5000,  3.5000,  5.0000, 63.0000, 62.5000],
          [69.5000, 23.

In [99]:
torch.max(x, axis=1)[0].unsqueeze(1).shape

torch.Size([2, 1, 5, 5])

In [103]:
class TMax(torch.nn.Module):
  def forward(self, x):
    return torch.max(x, axis=1)[0].unsqueeze(1)

In [104]:
m= TMax()
m(x)

tensor([[[[59.0000, 31.0000, 57.5000, 56.5000, 58.5000],
          [65.5000, 39.5000, 53.5000, 64.0000, 72.5000],
          [49.5000, 67.5000, 46.5000, 51.0000, 61.5000],
          [61.0000, 70.0000, 35.5000, 60.0000, 69.0000],
          [65.0000, 38.5000, 68.0000, 70.5000, 72.0000]]],


        [[[68.5000, 73.5000, 67.0000, 26.5000, 66.0000],
          [31.5000, 51.5000, 57.0000, 66.5000, 62.5000],
          [69.5000, 58.0000, 71.0000, 71.5000, 24.5000],
          [16.5000, 64.5000, 56.0000, 48.0000, 47.5000],
          [40.0000, 74.0000, 74.5000, 53.0000, 52.5000]]]])